In [44]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import *
spark=SparkSession.builder.appName('data_processing').getOrCreate()

Dans certaines mesures, vous auriez besoin d'appliquer des fonctions personnalisées.
Pour ce faire, nous allons utiliser une fonction de pyspark.sql.functions nommée udf().

Le fichier utilisé dans cette partie contient des adresses mails qui sont actifs ou inactifs.

In [91]:
users_email_csv_file = spark.read.csv("users_email.csv", header=True, inferSchema=True)

def verify_status(status):
    """
        Cette fonction permet de trier les utilisateurs actifs ou non 
        :param status: qui est le status actuel de l'adresse mail
        :return 'Compte actif' si status équivaut à 'Yes' ou 'Compte inactif' dans le cas contraire
    """
    if(status == 'Yes'):
        return "Compte actif"
    elif(status == 'No'):
        return "Compte inactif"

#definition de la structure de notre fonction udf qui prend en parametre 
# notre fonction personalisée et le type du parametre de la fonction
email_udf = udf(verify_status, StringType()) 

# ajout d'une colone 'isActive_text' sur lequel sera appliqué notre fonction email_udf
users_email_csv_file = users_email_csv_file.withColumn('isActive_text', email_udf(users_email_csv_file['isactive']))
users_email_csv_file.select(['email','isActive_text']).show()

+--------------------+--------------+
|               email| isActive_text|
+--------------------+--------------+
|sollicitudin.comm...|Compte inactif|
|senectus.et@proto...|  Compte actif|
|lacus.varius.et@i...|  Compte actif|
|    diam@hotmail.net|  Compte actif|
|   augue@outlook.net|Compte inactif|
|non.cursus@outloo...|  Compte actif|
|massa.integer.vit...|Compte inactif|
|ullamcorper.duis@...|  Compte actif|
| sem.eget@google.org|Compte inactif|
|orci.lacus@hotmai...|Compte inactif|
|et.magnis.dis@goo...|  Compte actif|
|mollis.phasellus@...|Compte inactif|
|eget.laoreet@hotm...|Compte inactif|
|pellentesque.ut@g...|Compte inactif|
|    proin@hotmail.ca|  Compte actif|
| semper@outlook.couk|  Compte actif|
|     nam@outlook.com|  Compte actif|
|    sed.pede@aol.com|  Compte actif|
|dui.semper@outloo...|Compte inactif|
|ac.fermentum@yaho...|  Compte actif|
+--------------------+--------------+
only showing top 20 rows



In [92]:
users_email_csv_file.groupBy('isActive_text').count().show() # compte le nombre de comptes actifs et inactifs 

+--------------+-----+
| isActive_text|count|
+--------------+-----+
|  Compte actif|  136|
|Compte inactif|  164|
+--------------+-----+

